import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when
#import findspark
#findspark.init()

create local SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark Basics") \
    .getOrCreate()

In [3]:
file_path = './ds_salaries.csv'

read csv with inferschema

In [4]:
df = spark.read.csv(file_path, header=True, inferSchema=True)

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [5]:
df = spark.read.csv(file_path, header=True, inferSchema=True)

write schema of scv on screen

In [6]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [7]:
schema = StructType([
    StructField(name="id", dataType=IntegerType(), nullable=True),
    StructField(name="work_year", dataType=IntegerType(), nullable=True),
    StructField(name="experience_level", dataType=StringType(), nullable=True),
    StructField(name="employment_type", dataType=StringType(), nullable=True),
    StructField(name="job_title", dataType=StringType(), nullable=True),
    StructField(name="salary", dataType=IntegerType(), nullable=True),
    StructField(name="salary_currency", dataType=StringType(), nullable=True),
    StructField(name="salary_in_usd", dataType=IntegerType(), nullable=True),
    StructField(name="employee_residence", dataType=StringType(), nullable=True),
    StructField(name="remote_ratio", dataType=IntegerType(), nullable=True),
    StructField(name="company_location", dataType=StringType(), nullable=True),
    StructField(name="company_size", dataType=StringType(), nullable=True)
])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [6]:
df = spark.read.csv(file_path, header=True, schema=schema)

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [6]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [8]:
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [8]:
display(df.toPandas())

Py4JError: An error occurred while calling o39.pandasStructHandlingMode. Trace:
py4j.Py4JException: Method pandasStructHandlingMode([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:321)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:329)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.lang.Thread.run(Thread.java:748)



create df_job_title that consists from all job_titles without duplicates

In [8]:
df_job_title = df.select("job_title").distinct()

print all rows from df_job_titles without truncating jobs

In [10]:
df_job_title.show(truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
+----------

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [9]:
df_analytic = df.groupBy("job_title").agg(
avg("salary_in_usd").alias("avg_salary"),
min("salary_in_usd").alias("min_salary"),
max("salary_in_usd").alias("max_salary")
)

print all rows from df_analytic without trancating jobs

In [13]:
df_analytic.show(truncate=False)

+----------------------------------------+------------------+----------+----------+
|job_title                               |avg_salary        |min_salary|max_salary|
+----------------------------------------+------------------+----------+----------+
|3D Computer Vision Researcher           |5409.0            |5409      |5409      |
|Lead Data Engineer                      |139724.5          |56000     |276000    |
|Head of Machine Learning                |79039.0           |79039     |79039     |
|Data Specialist                         |165000.0          |165000    |165000    |
|Data Analytics Lead                     |405000.0          |405000    |405000    |
|Machine Learning Scientist              |158412.5          |12000     |260000    |
|Lead Data Analyst                       |92203.0           |19609     |170000    |
|Data Engineering Manager                |123227.2          |59303     |174000    |
|Staff Data Scientist                    |105000.0          |105000    |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [10]:
df_sorted = df_analytic.orderBy(df_analytic.avg_salary.desc())
window_spec = Window.orderBy(df_sorted.avg_salary.desc())
df_analytic = df_sorted.withColumn("row_id", row_number().over(window_spec))

print all data from df_analytic

In [13]:
df_analytic.show(truncate=False)

+----------------------------------+------------------+----------+----------+------+
|job_title                         |avg_salary        |min_salary|max_salary|row_id|
+----------------------------------+------------------+----------+----------+------+
|Data Analytics Lead               |405000.0          |405000    |405000    |1     |
|Principal Data Engineer           |328333.3333333333 |185000    |600000    |2     |
|Financial Data Analyst            |275000.0          |100000    |450000    |3     |
|Principal Data Scientist          |215242.42857142858|148261    |416000    |4     |
|Director of Data Science          |195074.0          |130026    |325000    |5     |
|Data Architect                    |177873.9090909091 |90700     |266400    |6     |
|Applied Data Scientist            |175655.0          |54238     |380000    |7     |
|Analytics Engineer                |175000.0          |135000    |205300    |8     |
|Data Specialist                   |165000.0          |165000    

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [11]:
df_analytic = df_analytic.select("row_id", "job_title", "avg_salary", "min_salary", "max_salary")

print df_analytic now

In [15]:
df_analytic.show(truncate=False)

+------+----------------------------------+------------------+----------+----------+
|row_id|job_title                         |avg_salary        |min_salary|max_salary|
+------+----------------------------------+------------------+----------+----------+
|1     |Data Analytics Lead               |405000.0          |405000    |405000    |
|2     |Principal Data Engineer           |328333.3333333333 |185000    |600000    |
|3     |Financial Data Analyst            |275000.0          |100000    |450000    |
|4     |Principal Data Scientist          |215242.42857142858|148261    |416000    |
|5     |Director of Data Science          |195074.0          |130026    |325000    |
|6     |Data Architect                    |177873.9090909091 |90700     |266400    |
|7     |Applied Data Scientist            |175655.0          |54238     |380000    |
|8     |Analytics Engineer                |175000.0          |135000    |205300    |
|9     |Data Specialist                   |165000.0          |165

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [12]:
window_spec = Window.partitionBy("experience_level").orderBy(df.salary_in_usd.desc())
df_with_rank = df.withColumn("rank", row_number().over(window_spec))
df_exp_lvl = df_with_rank.filter(df_with_rank.rank == 1).drop("rank")

print here df_exp_lvl

In [17]:
df_exp_lvl.show(truncate=False)

+---+---------+----------------+---------------+-------------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|id |work_year|experience_level|employment_type|job_title                |salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+-------------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|37 |2020     |EN              |FT             |Machine Learning Engineer|250000|USD            |250000       |US                |50          |US              |L           |
|252|2021     |EX              |FT             |Principal Data Engineer  |600000|USD            |600000       |US                |100         |US              |L           |
|33 |2020     |MI              |FT             |Research Scientist       |450000|USD            |450000       |US                |

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [13]:
window_spec = Window.partitionBy("experience_level")
df_with_max_salary = df.withColumn("biggest_salary", max("salary_in_usd").over(window_spec))
df_best = df_with_max_salary.filter(col("salary_in_usd") == col("biggest_salary"))
df_best = df_best.select("id", "experience_level", "biggest_salary", "employee_residence")

print df_best

In [21]:
df_best.show(truncate=False)

+---+----------------+--------------+------------------+
|id |experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
|37 |EN              |250000        |US                |
|252|EX              |600000        |US                |
|33 |MI              |450000        |US                |
|97 |MI              |450000        |US                |
|63 |SE              |412000        |US                |
+---+----------------+--------------+------------------+



drop duplicates if exist by experience_level

In [14]:
df_best = df_best.dropDuplicates(["experience_level"])

print df_best

In [23]:
df_best.show()

+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|              EN|        250000|                US|
|252|              EX|        600000|                US|
| 33|              MI|        450000|                US|
| 63|              SE|        412000|                US|
+---+----------------+--------------+------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [15]:
df_new_best = df_best.drop("id")

df_new_best = df_new_best.withColumn(
"experience_level",
when(df_new_best.experience_level == "MI", "middle")
.when(df_new_best.experience_level == "SE", "senior")
.otherwise(None)
)

print df_new_best

In [25]:
df_new_best.show()

+----------------+--------------+------------------+
|experience_level|biggest_salary|employee_residence|
+----------------+--------------+------------------+
|            null|        250000|                US|
|            null|        600000|                US|
|          middle|        450000|                US|
|          senior|        412000|                US|
+----------------+--------------+------------------+



In [16]:
output_path = "./1.csv"

write df_new_best like 1.csv and load then it to df_final

In [17]:
df_new_best.write.csv(output_path, header=True, mode="overwrite")
df_final = spark.read.csv("1.csv", header=True, inferSchema=True)

print df_final

In [23]:
df_final.show()

+----------------+--------------+------------------+
|experience_level|biggest_salary|employee_residence|
+----------------+--------------+------------------+
|            null|        250000|                US|
|            null|        600000|                US|
|          middle|        450000|                US|
|          senior|        412000|                US|
+----------------+--------------+------------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [18]:
df_final_filtered = df_final.filter(df_final.experience_level.isNotNull())

df_final = df_final_filtered.join(df, 
(df_final_filtered.biggest_salary == df.salary_in_usd) & 
(df_final_filtered.employee_residence == df.employee_residence), how="inner")

print df_final

In [25]:
df_final.show()

+----------------+--------------+------------------+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|experience_level|biggest_salary|employee_residence| id|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+----------------+--------------+------------------+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|          middle|        450000|                US| 33|     2020|              MI|             FT|  Research Scientist|450000|            USD|       450000|                US|           0|              US|           M|
|          senior|        412000|                US| 63|     2020|              SE|             FT|      Data Scientist|

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [19]:
max_salary = df_final.agg({"salary_in_usd": "max"}).collect()[0][0]

print(f"The biggest salary_in_usd in df_final is: {max_salary}")

The biggest salary_in_usd in df_final is: 450000


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system